# Part 1: Processing

The following converts all the Excel sheets of predictions into one DataFrame complete with all information needed to evaluate the Euro 2020 bracket entries. The final standings produced by this can be viewed at https://aaronmoniz.com/projects/euro-2020-bracket-standings/

In [34]:
# Importing necessary libraries
import pandas as pd
import glob

In [35]:
def opponent_finder(l1, l2):
    # args: two lists l1 and l2 containing country names
    # returns: a populated list of opponents for each round by country
    x = []
    for i in list(aaron2['country']):
        if i in l2:
            x += [l1[l2.index(i)]]
        elif i in l1:
            x += [l2[l1.index(i)]]
        else:
            x += ['']
    return x

In [36]:
def no_fours(l):
    # arg: series l of group rankings by country
    # returns: a populated list of group ranks in case the 4 has been omitted by the user
    g = list(l)
    if 4 not in g:
        p = [g[1], g[2], g[3], g[4], g[7], g[8], g[9], g[10], g[13], g[14], g[15], g[16],\
             g[19], g[20], g[21], g[22], g[25], g[26], g[27], g[28], g[31], g[32], g[33], g[34] ]
        r = []
        for i in p:
            if type(i) == float:
                r += [4]
            else:
                r += [i]
        return r
    else:
        r = [rank for rank in list(aaron['Unnamed: 1']) \
                            if pd.isnull(rank) == False and type(rank) == int][:24]
        return r

In [ ]:
# reads through list of bracket entries in a folder and converts them into one DataFrame, where
# each row represents the predictions one user made about one country
entries = pd.DataFrame()
for c in glob.glob(r"C:\Users\aaron\Documents\Euro 2020 Bracket Sample\*.xlsx"):
    bracket = pd.read_excel(c)
    bracket2 = pd.DataFrame() # resets DataFrame to empty
    # the following lines read through the sheet for important cells and format them in order
    # to populate the DataFrame
    bracket2['country'] = [country for country in list(bracket['Unnamed: 2']) \
                        if pd.isnull(country) == False and type(country) == str][:24]
    bracket2['group rank'] = no_fours(bracket['Unnamed: 1'][11:46])
    bracket2['qualify from group?'] = [country in list(bracket['Unnamed: 6']) \
                                     for country in bracket2['country']]
    l = list(bracket['Unnamed: 6'])
    l2 = [l[13], l[17], l[21], l[25], l[29], l[33], l[37], l[41]]
    l3 = [l[15], l[19], l[23], l[27], l[31], l[35], l[39], l[43]]
    l4 = list(bracket['Unnamed: 9'])
    l5 = [l4[15], l4[23], l4[31], l4[39]]
    l6 = [l4[17], l4[25], l4[33], l4[41]]
    l7 = list(bracket['Unnamed: 11'])
    l8 = [l7[19], l7[35]]
    l9 = [l7[21], l7[37]]
    l10 = list(bracket['Unnamed: 13'])
    s11 = [l10[27]]
    s12 = [l10[29]]
    s13 = [l10[23]]
    # The following lines take the processed data from above and load them into the DataFrame
    # and standardizes their format
    bracket2['r16 opponent'] = opponent_finder(l2,l3)
    bracket2['qf opponent'] = opponent_finder(l5, l6)
    bracket2['sf opponent'] = opponent_finder(l8,l9)
    bracket2['final opponent'] = opponent_finder(s11,s12)
    bracket2['winner'] = opponent_finder(s13,s13)
    bracket2['Twitter'] = bracket['Unnamed: 11'][5]
    bracket2['name'] = bracket['Unnamed: 11'][4]
    bracket2['Twitter'] = [handle.replace('@','') for handle in \
                           [str(h) for h in list(bracket2['Twitter'])]]
    entries = pd.concat([entries, bracket2]) # appends the processed entry to the others

In [ ]:
# saves output of previous cell to CSV
entries.to_csv(r'C:\Users\aaron\Documents\BracketResults.csv')
entries